In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
pip install mlflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /usr/local/python/3.12.1/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install boto3


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /usr/local/python/3.12.1/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
print(boto3.__version__)

1.37.17


In [18]:
import boto3

# Create a session using the profile 'ml_user'
session = boto3.Session(profile_name='ml_user')

# Create an S3 client using the session
s3_client = session.client('s3')

# List objects in the specified S3 bucket
response = s3_client.list_objects_v2(Bucket='mlflow-models-slv')

# Print the response
print(response)

{'ResponseMetadata': {'RequestId': 'FWPW5B0SYW3ZFVTA', 'HostId': 'ooyHoQp6Em1MfFDQbs2BpYlfJsARCYQACrkQCzFlmLzOtHCcSMe79XMsojGMUWQZlKGJWXLwkZM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ooyHoQp6Em1MfFDQbs2BpYlfJsARCYQACrkQCzFlmLzOtHCcSMe79XMsojGMUWQZlKGJWXLwkZM=', 'x-amz-request-id': 'FWPW5B0SYW3ZFVTA', 'date': 'Fri, 21 Mar 2025 16:20:07 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Name': 'mlflow-models-slv', 'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 0}


In [ ]:
boto3.client(
            's3',
            endpoint_url=config['s3']['endpoint_url'],
            aws_access_key_id=config['s3']['access_key_id'],
            aws_secret_access_key=config['s3']['secret_access_key'],
            region_name=config['s3']['region_name']
        )

In [22]:
import boto3

# Create an S3 client using the credentials and region from AWS CLI configuration
s3_client = boto3.client('s3')


In [ ]:
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
AWS_DEFAULT_REGION = os.environ['AWS_DEFAULT_REGION']

In [6]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green taxi duration")

<Experiment: artifact_location='s3://mlflow-models-slv/1', creation_time=1742505694391, experiment_id='1', last_update_time=1742505694391, lifecycle_stage='active', name='green taxi duration', tags={}>

In [7]:
def read_dataframe(filename: str):
    df=pd.read_parquet(filename)
    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda m : m.total_seconds() / 60)
    
    df=df[(df.duration >= 1) & (df.duration <= 60)]
    categorical=['PULocationID','DOLocationID']
    df[categorical]=df[categorical].astype(str)
    return df    
    

In [8]:
def prepare_dictionaries(df:pd.DataFrame):
    df['PU_DO']=df['PULocationID'] + '_' + df['DOLocationID']
    categorical=['PU_DO']
    numerical=['trip_distance']
    dicts=df[categorical + numerical].to_dict(orient='records')
    return dicts

In [9]:
df_train=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

dict_train=prepare_dictionaries(df_train)
dict_val=prepare_dictionaries(df_val)

In [10]:
len(df_train),len(df_val)

(73908, 61921)

In [23]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params,n_jobs=-1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train,y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

   
    mse = mean_squared_error(y_val, y_pred)
    print(params,mse)
    mlflow.log_metric('mse',mse)
    
    mlflow.sklearn.log_model(model,"model")

    with open('dict_vectorizer.bin','wb') as f_out:
        pickle.dump(dv,f_out)
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588


2025/03/21 16:45:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run entertaining-ox-488 at: http://127.0.0.1:5000/#/experiments/1/runs/a55d30dade9943ed8c8395ee96f85830
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


NoCredentialsError: Unable to locate credentials

In [21]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)
    pipeline=make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params,n_jobs=-1)
    )
    pipeline.fit(dict_train,y_train)
    y_pred=pipeline.predict(dict_val)
    
    mse=mean_squared_error(y_pred,y_val)
    print(params,mse)
    mlflow.log_metric('mse',mse)
    mlflow.sklearn.log_model(pipeline,artifact_path="model")
    
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588


2025/03/21 16:32:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run merciful-sow-879 at: http://127.0.0.1:5000/#/experiments/1/runs/7bc884c5de38489e8d91580432d2dc83
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


NoCredentialsError: Unable to locate credentials

In [6]:
from mlflow.tracking import MlflowClient

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
MLFLOW_TRACKING_URI = ''
RUN_ID = ''
client=MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)

In [ ]:
path=client.download_artifacts(run_id=RUN_ID,path='dict_vectorizer.bin')

In [ ]:
with open(path,'rb')as f_out:
    dv=pickle.load(f_out)
          